# xg1test

In [1]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
from functools import partial, reduce
import logging

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname

def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fname = 'dataload_test.ipynb'   # FILL
dir_name = 'model'              # FILL
fix_path(get_cwd(fname, dir_name +sep))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dask import delayed, compute
from torch.utils.data import TensorDataset, DataLoader
import torch

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option('display.max_columns', 50)

from common_util import RECON_DIR, JSON_SFX_LEN, DT_CAL_DAILY_FREQ, is_type, pd_common_idx_rows, remove_dups_list, set_loglevel, search_df, chained_filter, get_variants, dump_df, load_json, gb_transpose, pd_common_index_rows, filter_cols_below, inner_join, outer_join, ser_shift, list_get_dict, window_iter, benchmark
from common_util import midx_get_level, midx_intersect, pd_common_idx_rows, midx_split, pd_midx_to_arr, window_iter, np_is_ndim
from model.common import DATASET_DIR, XG_DIR, EXPECTED_NUM_HOURS, default_dataset
from model.data_util import xgdg, align_first_last_cols, prune_nulls
from data.data_api import DataAPI
from recon.dataset_util import prep_dataset, gen_group
from recon.split_util import get_train_test_split, gen_time_series_split, index_three_split, pd_binary_clip

Using TensorFlow backend.


### Settings

In [2]:
set_loglevel('info')

In [3]:
assets_str = 'sp_500'
assets = list(map(str.strip, assets_str.split(',')))

### X

In [4]:
xg1_dict = load_json('xg1_mkt.json', dir_path=DATASET_DIR)
xg1_dataset = prep_dataset(xg1_dict, assets=assets, filters_map=None)

In [5]:
xg1_dataset['features']['recs']

[['sp_500', 'dmx', 'raw_pba'], ['sp_500', 'dmx', 'raw_vol'], ['sp_500', 'dzn', 'raw_pba'], ['sp_500', 'dzn', 'raw_vol'], ['sp_500', 'dmx_uni', 'raw_pba', 'raw_pba_dmx_uni(2)'], ['sp_500', 'dmx_uni', 'raw_pba', 'raw_pba_dmx_uni(3)'], ['sp_500', 'dmx_uni', 'raw_pba', 'raw_pba_dmx_uni(4)'], ['sp_500', 'dmx_uni', 'raw_pba', 'raw_pba_dmx_uni(8)'], ['sp_500', 'dmx_uni', 'raw_vol', 'raw_vol_dmx_uni(2)'], ['sp_500', 'dmx_uni', 'raw_vol', 'raw_vol_dmx_uni(3)'], ['sp_500', 'dmx_uni', 'raw_vol', 'raw_vol_dmx_uni(4)'], ['sp_500', 'dmx_uni', 'raw_vol', 'raw_vol_dmx_uni(8)'], ['sp_500', 'dzn_gau', 'raw_pba', 'raw_pba_dzn_gau(2)'], ['sp_500', 'dzn_gau', 'raw_pba', 'raw_pba_dzn_gau(3)'], ['sp_500', 'dzn_gau', 'raw_pba', 'raw_pba_dzn_gau(4)'], ['sp_500', 'dzn_gau', 'raw_pba', 'raw_pba_dzn_gau(8)'], ['sp_500', 'dzn_gau', 'raw_vol', 'raw_vol_dzn_gau(2)'], ['sp_500', 'dzn_gau', 'raw_vol', 'raw_vol_dzn_gau(3)'], ['sp_500', 'dzn_gau', 'raw_vol', 'raw_vol_dzn_gau(4)'], ['sp_500', 'dzn_gau', 'raw_vol', 'raw_v

### Test Loop

In [7]:
for xg_fname in test_xgs:
    logging.info('xg: {}'.format(xg_fname))

    for i, (paths, recs, dfs) in enumerate(xgdg(xg_fname, delayed=True, assets=assets, filters_map=None)):
        fpath, lpath, tpath = paths
        frec, lrec, trec = recs
        logging.info('(X, y, z) -> ({fdesc}[:], {ldesc}[:], {tdesc}[:])'.format(fdesc=frec.desc, ldesc=lrec.desc, tdesc=trec.desc))
        with benchmark('time to load data') as b:
            f, l, t = dfs.compute()

        val_ratio = .2
        test_ratio = .2
        train_ratio = 1-(val_ratio+test_ratio)
        f_train_idx, f_val_idx, f_test_idx = midx_split(f.index, train_ratio, val_ratio, test_ratio)
        l_train_idx, l_val_idx, l_test_idx = midx_split(l.index, train_ratio, val_ratio, test_ratio)
        t_train_idx, t_val_idx, t_test_idx = midx_split(t.index, train_ratio, val_ratio, test_ratio)

        f_train_pd, f_val_pd, f_test_pd = f.loc[f_train_idx], f.loc[f_val_idx], f.loc[f_test_idx]
        l_train_pd, l_val_pd, l_test_pd = l.loc[l_train_idx], l.loc[l_val_idx], l.loc[l_test_idx]
        t_train_pd, t_val_pd, t_test_pd = t.loc[t_train_idx], t.loc[t_val_idx], t.loc[t_test_idx]

        if (is_type(f.index, pd.core.index.MultiIndex)):
            f_train_np, f_val_np, f_test_np = map(pd_midx_to_arr, [f_train_pd.stack(), f_val_pd.stack(), f_test_pd.stack()])
        else:
            f_train_np, f_val_np, f_test_np = f_train_pd.values, f_val_pd.values, f_test_pd.values
        l_train_np, l_val_np, l_test_np = l_train_pd.values, l_val_pd.values, l_test_pd.values
        t_train_np, t_val_np, t_test_np = t_train_pd.values, t_val_pd.values, t_test_pd.values

        display('f.shape: {}, l.shape:{}, t.shape:{}'.format(f_train_np.shape, l_train_np.shape, t_train_np.shape))
        display('f:', f_train_np)

        if (first_only):
            break

INFO:root:xg: test_xg0_single_channel_daily.json
INFO:root:(X, y, z) -> (raw_pba_oc_retxeod_reteod[:], raw_pba_oc_retxeod_direod[:], raw_pba_oc_retxeod_reteod[:])


'f.shape: (3007, 1), l.shape:(3007,), t.shape:(3007,)'

'f:'

array([[ 0.00470925],
       [ 0.00208197],
       [-0.01073618],
       ...,
       [ 0.0039105 ],
       [ 0.00089008],
       [-0.00663572]])

INFO:root:xg: test_xg1_multi_channel_daily.json
INFO:root:(X, y, z) -> (raw_trmi_v2_drl[:], raw_pba_oc_retxeod_direod[:], raw_pba_oc_retxeod_reteod[:])


'f.shape: (3007, 31), l.shape:(3007,), t.shape:(3007,)'

'f:'

array([[-9.92680e-02, -8.99600e-03,  7.75500e-03, ...,  7.76000e-04,
         2.32700e-03,  1.47350e-02],
       [-9.74290e-02, -2.22300e-03,  4.44600e-03, ...,  9.67000e-04,
        -9.70000e-05,  1.15990e-02],
       [-1.10931e-01, -2.24380e-02,  7.02900e-03, ...,  3.60000e-04,
         9.91000e-04,  1.38780e-02],
       ...,
       [-5.45210e-02, -7.15100e-03,  7.10500e-03, ...,  7.35000e-04,
         5.00000e-06,  4.81000e-03],
       [-3.58700e-03,  1.24880e-02,  5.20500e-03, ...,  4.13000e-04,
         1.09000e-03,  2.57200e-03],
       [-1.59867e-01, -1.26830e-02,  7.02500e-03, ...,  9.56000e-04,
         1.80000e-05,  3.85800e-03]])

INFO:root:xg: test_xg2_single_channel_intraday.json
INFO:root:(X, y, z) -> (raw_pba_dmx[:], raw_pba_oc_retxeod_direod[:], raw_pba_oc_retxeod_reteod[:])


ValueError: Inferred frequency None from passed values does not conform to passed frequency D

## Multi Index Row Filtering

In [37]:
for i, (paths, recs, dfs) in enumerate(xgdg(test_xgs[-1], delayed=True, assets=assets, filters_map=None)):
    fpath, lpath, tpath = paths
    frec, lrec, trec = recs
    logging.info('(X, y, z) -> ({fdesc}[:], {ldesc}[:], {tdesc}[:])'.format(fdesc=frec.desc, ldesc=lrec.desc, tdesc=trec.desc))
    with benchmark('time to load data') as b:
        f, l, t = dfs.compute()

    val_ratio = .2
    test_ratio = .2
    train_ratio = 1-(val_ratio+test_ratio)
    f_train_idx, f_val_idx, f_test_idx = midx_split(f.index, train_ratio, val_ratio, test_ratio)
    l_train_idx, l_val_idx, l_test_idx = midx_split(l.index, train_ratio, val_ratio, test_ratio)
    t_train_idx, t_val_idx, t_test_idx = midx_split(t.index, train_ratio, val_ratio, test_ratio)

    f_train_pd, f_val_pd, f_test_pd = f.loc[f_train_idx], f.loc[f_val_idx], f.loc[f_test_idx]
    l_train_pd, l_val_pd, l_test_pd = l.loc[l_train_idx], l.loc[l_val_idx], l.loc[l_test_idx]
    t_train_pd, t_val_pd, t_test_pd = t.loc[t_train_idx], t.loc[t_val_idx], t.loc[t_test_idx]

    if (is_type(f.index, pd.core.index.MultiIndex)):
        f_train_np, f_val_np, f_test_np = map(pd_midx_to_arr, [f_train_pd.stack(), f_val_pd.stack(), f_test_pd.stack()])
    else:
        f_train_np, f_val_np, f_test_np = f_train_pd.values, f_val_pd.values, f_test_pd.values
    l_train_np, l_val_np, l_test_np = l_train_pd.values, l_val_pd.values, l_test_pd.values
    t_train_np, t_val_np, t_test_np = t_train_pd.values, t_val_pd.values, t_test_pd.values

    display('f.shape: {}, l.shape:{}, t.shape:{}'.format(f_train_np.shape, l_train_np.shape, t_train_np.shape))
    display('f:', f_train_np)

INFO:root:(X, y, z) -> (raw_pba_dmx[:], raw_pba_oc_retxeod_direod[:], raw_pba_oc_retxeod_reteod[:])


'f.shape: (2989, 5, 8), l.shape:(2989,), t.shape:(2989,)'

'f:'

array([[[ 0.2532595 , -0.79730638, -0.42622113, ..., -1.        ,
         -0.38699965,  1.        ],
        [-0.04369538, -0.360799  , -0.50561798, ..., -1.        ,
          0.60799001,  1.        ],
        [ 0.65068493, -0.47945205, -0.44520548, ..., -1.        ,
          0.45890411,  1.        ],
        [ 0.20512821, -0.93846154, -0.18205128, ..., -1.        ,
         -0.65128205,  1.        ],
        [ 0.03726708,  0.12732919, -0.21118012, ..., -0.35714286,
         -1.        ,  1.        ]],

       [[-0.06457293,  0.50353947,  0.91668164, ..., -1.        ,
         -0.53911387,  0.17817145],
        [-0.59699625,  0.19148936,  1.        , ..., -1.        ,
         -0.54443054, -0.27659574],
        [-0.19610231,  0.47868453,  1.        , ..., -1.        ,
         -0.58099878, -0.34957369],
        [ 0.12549801,  0.37250996,  0.98804781, ..., -1.        ,
         -0.70717131,  0.38247012],
        [-0.78419072, -0.57089084,  0.20953576, ..., -0.95483061,
         -1.  

In [57]:
date_range = {
    'id0': ('ine', 2009, 2018),
}

In [60]:
f.loc[search_df(f, date_range)]

8         9         10        11        12  \
id0        id1                                                              
2009-01-02 pba_avgPrice -1.000000 -0.688267 -0.093641  0.065233  0.360852   
           pba_close    -1.000000 -0.058166  0.020046  0.192902  0.493920   
           pba_high     -1.000000 -0.330963 -0.113402  0.058656  0.360114   
           pba_low      -1.000000 -0.948160 -0.062400  0.088000  0.295360   
           pba_open     -0.854887 -1.000000  0.006382  0.079610  0.261673   
2009-01-05 pba_avgPrice -1.000000  0.147637  0.597856  0.597196  1.000000   
           pba_close    -1.000000  0.517289 -0.045643  0.286307  1.000000   
           pba_high     -0.682074  0.312289  0.032694  0.210823  0.797069   
           pba_low      -1.000000 -0.825630  0.596639  0.550420  1.000000   
           pba_open      0.181757 -1.000000  0.427230 -0.098592  0.278337   
2009-01-06 pba_avgPrice  1.000000 -0.179108 -1.000000 -0.100052 -0.505435   
           pba_close     1.000000 -1.000000 -0.895105 -0.043706 -0.667832   
           pba_high      0.939394  1.000000 -1.000000 -0.253788 -0.431818   
           pba_low       0.089636 -0.436975 -1.000000  0.056022 -0.336134   
           pba_open     -1.000000  1.000000 -0.996845 -0.929022 -0.231861   
2009-01-07 pba_avgPrice  1.000000  0.432083  0.664829  0.569891  0.272498   
           pba_close     1.000000  0.252762  0.915746  0.099448 -0.480663   
           pba_high      1.000000  0.460348  0.358801  0.264990 -0.002901   
           pba_low       1.000000  0.452646  0.590529  0.565460  0.001393   
           pba_open      1.000000  0.405133 -0.170486  0.303391 -0.232814   
2009-01-08 pba_avgPrice -0.787476 -0.655063 -0.343317 -0.504110 -1.000000   
           pba_close    -1.000000  0.118525 -0.757682 -0.385426 -0.683933   
           pba_high     -0.033898  0.162228  0.428571 -0.217918 -1.000000   
           pba_low      -0.941131 -1.000000 -0.679318 -0.563129 -0.897754   
           pba_open      0.255487 -1.000000  0.085162 -0.792801 -0.445127   
2009-01-09 pba_avgPrice  1.000000 -0.237007  0.019196  0.016775 -0.184904   
           pba_close    -0.113025  0.001076  0.214209  0.063509 -0.158235   
           pba_high      1.000000 -0.393883 -0.210380 -0.220575 -0.324374   
           pba_low       0.078853 -0.182796  0.777778  0.996416  0.448029   
           pba_open      1.000000 -0.671357 -0.564824 -0.447236 -0.505528   
2009-01-12 pba_avgPrice  1.000000  0.572980  0.136661 -0.070703 -0.050502   
           pba_close     1.000000  0.528390 -0.233133 -0.221109 -0.162325   
           pba_high      1.000000  0.319762  0.020854 -0.301887 -0.270109   
           pba_low       1.000000  0.605327  0.200969  0.106538  0.174334   
           pba_open      1.000000  0.377838  0.056312 -0.478656 -0.447775   
2009-01-13 pba_avgPrice  0.014447  0.600446  1.000000  0.903280 -0.880570   
           pba_close     0.675241  0.715434  1.000000  0.213826 -1.000000   
           pba_high      0.375912  0.981752  1.000000  0.784672 -1.000000   
           pba_low      -0.360529  0.404631  0.750827  0.955899 -0.834620   
           pba_open     -0.012442  0.636081  0.597201  1.000000  0.132193   
2009-01-14 pba_avgPrice  1.000000 -0.374908 -0.959997 -0.774534 -0.707011   
           pba_close     1.000000 -1.000000 -0.367370 -0.034406 -0.704772   
           pba_high      1.000000 -0.286935 -0.842898 -0.768010 -0.734636   
           pba_low       1.000000 -0.814377 -1.000000 -0.501365 -0.444950   
           pba_open      1.000000 -0.353581 -1.000000 -0.763780 -0.652043   
2009-01-15 pba_avgPrice  0.228949 -0.635879 -0.625583 -1.000000 -0.554181   
           pba_close    -0.047358 -0.756556 -0.671233 -1.000000  0.228180   
           pba_high      0.268405 -0.338957 -0.832055 -1.000000 -0.053681   
           pba_low      -0.108189 -0.778362 -0.577761 -1.000000 -0.725019   
           pba_open      0.731976  0.036253 -0.805295 -0.694501 -1.000000   
...                        

In [62]:
f.loc[search_df(f, date_range)].values.shape

(11315, 8)

In [63]:
f.values.shape

(24910, 8)